In [1]:
from igraph import *
import pandas as pd
import numpy as np

In [4]:
import os
cwd = os.getcwd()

covs = pd.read_csv('C:/users/5SQQJ72/Documents/leuphana/semester_2/network_analysis/group_project/cover_data_files/covRelations.csv')
del covs['Unnamed: 0']

covs['ones'] = 1

In [5]:
covs_count = covs.groupby('org_art_id')['ones'].sum()
covs_count = covs_count[covs_count > 3].reset_index()

In [6]:
covs_rev = covs[covs['org_art_id'].isin(covs_count['org_art_id'])]

made_covs_count = covs.groupby('cov_art_id')['ones'].sum()
made_covs_count = made_covs_count[made_covs_count > 1].reset_index()

covs_rev_2 = covs_rev[covs_rev['cov_art_id'].isin(made_covs_count['cov_art_id'])]
covs_rev_2['release_decade'] = (covs_rev_2['org_release_year'].astype('str').str[:3] + '0').astype(int)
covs_rev_2['perf_decade'] = (covs_rev_2['perf_year'].astype('str').str[:3] + '0').astype(int)

covs_rev_2 = covs_rev_2[(covs_rev_2.release_decade >= 1960) & (covs_rev_2.release_decade <= 1995)]

orgs1960 = covs_rev_2[covs_rev_2.release_decade == 1960]
orgs1970 = covs_rev_2[covs_rev_2.release_decade == 1970]
orgs1980 = covs_rev_2[covs_rev_2.release_decade == 1980]

covs1990 = covs_rev_2[covs_rev_2.perf_decade == 1990]

unique1980 = orgs1980['org_art_id'].unique()

covs1990 = covs1990[covs1990.org_art_id.isin(unique1980)]

C:\Users\5SQQJ72\Miniconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\5SQQJ72\Miniconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
arts = orgs1980[['org_art_id','cov_art_id']].reset_index()
arts = arts.iloc[:,1:]
arts_unique = arts.groupby(['org_art_id','cov_art_id']).size().reset_index()
arts_unique = arts_unique.iloc[:,0:2]

In [8]:
vertices = pd.concat([arts['org_art_id'], arts['cov_art_id']], axis=0).unique()
vertices = pd.DataFrame(vertices).reset_index()
vertices.columns = ['v_no','art_id']

In [9]:
edge1 = vertices.merge(arts, left_on = 'art_id', right_on = 'org_art_id', how='inner')
edge2 = vertices.merge(edge1, left_on = 'art_id', right_on = 'cov_art_id', how='right')
edge2 = edge2.iloc[:,:4]
edge2.columns = ['cov_v','cov_art_id','org_v','org_art_id']
edgelist = edge2[['cov_v','org_v']]

el = edgelist.values.tolist()

In [10]:
cov_graph = Graph(n=len(vertices), directed=True, vertex_attrs={'name': vertices.art_id})
#cov_graph.add_vertices(vertices)
cov_graph.add_edges(el)

In [11]:
n = cov_graph.vcount()
print('total vertices:', n)
neighbs = [v.neighbors('out') for v in cov_graph.vs]

total vertices: 11817


In [12]:
score_pa = np.zeros(shape = (n, n))
for i, x in enumerate(neighbs):
    for j, y in enumerate(neighbs):
        score_pa[i,j] = len(x) * len(y)

In [26]:
comm_neighbs = np.zeros(shape = (n,n))
for i, x in enumerate(neighbs):
    for j, y in enumerate(neighbs):
        comm_neighbs[i,j] = len(set(x) & set(y))

In [30]:
jaccard_coeff = np.zeros(shape = (n,n))
for i, x in enumerate(neighbs):
    for j, y in enumerate(neighbs):
        inters = (len( set(x) | set(y) ))
        if inters == 0:
            jaccard_coeff[i,j] = 0
        else:
            jaccard_coeff[i,j] = len(set(x) & set(y)) / (inters)

In [33]:
salton_cos = np.zeros(shape = (n, n))
for i, x in enumerate(neighbs):
    for j, y in enumerate(neighbs):
        cos = ( np.sqrt( len(x) * len(y) ) ) 
        if cos == 0:
            salton_cos[i,j] = 0
        else:
            salton_cos[i,j] = len(set(x) & set(y)) / (cos) 